In [1]:
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import keyring
import time

## Setup Spotipy credentials and query wrapper

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id=keyring.get_password('spotify', 'cid'),
                                                      client_secret=keyring.get_password('spotify', 'secret') )
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Get sample artists data

In [3]:
artist_id = '0du5cEVh5yTK9QJze8zA0C'

In [4]:
# View sp.track output
sp.artist(artist_id)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0du5cEVh5yTK9QJze8zA0C'},
 'followers': {'href': None, 'total': 26715041},
 'genres': ['dance pop', 'pop', 'post-teen pop'],
 'href': 'https://api.spotify.com/v1/artists/0du5cEVh5yTK9QJze8zA0C',
 'id': '0du5cEVh5yTK9QJze8zA0C',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/aa32d6d4ca2467974403518dd3ebfe8831c5ced1',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/63cad86cb183568085a4e5f0be86aa187cc511e4',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/5461dcbb3f23e1df62d85c8273fc49cc4edd49d5',
   'width': 160}],
 'name': 'Bruno Mars',
 'popularity': 89,
 'type': 'artist',
 'uri': 'spotify:artist:0du5cEVh5yTK9QJze8zA0C'}

## Get sample track data

In [5]:
track_id = "0ofbQMrRDsUaVKq2mGLEAb"

In [6]:
# View sp.track output
sp.track(track_id)

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
    'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
    'id': '4nDoRrQiYLoBzwC5BhVJzF',
    'name': 'Camila Cabello',
    'type': 'artist',
    'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/5chBPOVY2I0bG5V3igb5QL'},
  'href': 'https://api.spotify.com/v1/albums/5chBPOVY2I0bG5V3igb5QL',
  'id': '5chBPOVY2I0bG5V3igb5QL',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273d93cf4d07ba50d7b32ca7c44',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02d93cf4d07ba50d7b32ca7c44',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851d93cf4d07ba50d7b32ca7c44',
    'width': 64}],
  'name': 'Havana',
  'release_date': '2017-08-03',
  'release_date

In [7]:
# View sp.audio_featrues output
sp.audio_features(track_id)

[{'danceability': 0.768,
  'energy': 0.517,
  'key': 7,
  'loudness': -4.323,
  'mode': 0,
  'speechiness': 0.0312,
  'acousticness': 0.186,
  'instrumentalness': 3.8e-05,
  'liveness': 0.104,
  'valence': 0.418,
  'tempo': 104.992,
  'type': 'audio_features',
  'id': '0ofbQMrRDsUaVKq2mGLEAb',
  'uri': 'spotify:track:0ofbQMrRDsUaVKq2mGLEAb',
  'track_href': 'https://api.spotify.com/v1/tracks/0ofbQMrRDsUaVKq2mGLEAb',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0ofbQMrRDsUaVKq2mGLEAb',
  'duration_ms': 216897,
  'time_signature': 4}]

## Read consolidated spotify daily charts

In [8]:
df = pd.read_csv('data/spotify_daily_charts.csv')
df.head()

,date,position,track_id,track_name,artist,streams
0,2018-01-01,1,0ofbQMrRDsUaVKq2mGLEAb,Havana,Camila Cabello,155633
1,2018-01-01,2,0tgVpDi06FyKpA1z0VMD4v,Perfect,Ed Sheeran,134756
2,2018-01-01,3,3hBBKuWJfxlIlnd9QFoC8k,What Lovers Do (feat. SZA),Maroon 5,130898
3,2018-01-01,4,1mXVgsBdtIVeCLJnSnmtdV,Too Good At Goodbyes,Sam Smith,130798
4,2018-01-01,5,2ekn2ttSfGqwhhate0LSR0,New Rules,Dua Lipa,125472


## Get data of unique tracks in charts 

In [9]:
def get_track_data(t_id):                    
    track_data = sp.track(t_id)
    track_features = sp.audio_features(t_id)
    
    #get only main(first) artist
    td_list = [t_id,\
               track_data['name'],\
               track_data['artists'][0]['id'],\
               track_data['artists'][0]['name'],\
               track_data['album']['uri'].split(":")[2],\
               track_data['duration_ms'],\
               track_data['album']['release_date'],\
               track_data['popularity']]
    data = pd.DataFrame([td_list], columns = ['track_id','track_name','artist_id','artist_name','album_id','duration','release_date','popularity'])

    relevant_cols = ['danceability', 'energy', 'key', 'loudness', 'mode',\
                     'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']  
    
    tf_data = pd.DataFrame(track_features)
    tf_data = tf_data[relevant_cols]
    
    data = pd.concat([data, tf_data], axis=1)
    return data


In [10]:
get_track_data(track_id)

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0ofbQMrRDsUaVKq2mGLEAb,Havana,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,5chBPOVY2I0bG5V3igb5QL,216896,2017-08-03,5,0.768,0.517,7,-4.323,0,0.0312,0.186,0.000038,0.104,0.418,104.992


In [11]:
track_df = df[['track_id','track_name']].drop_duplicates()
track_df

,track_id,track_name
0,0ofbQMrRDsUaVKq2mGLEAb,Havana
1,0tgVpDi06FyKpA1z0VMD4v,Perfect
2,3hBBKuWJfxlIlnd9QFoC8k,What Lovers Do (feat. SZA)
3,1mXVgsBdtIVeCLJnSnmtdV,Too Good At Goodbyes
4,2ekn2ttSfGqwhhate0LSR0,New Rules
...,...,...
196985,0zzVTGyRrWpQu8Fr28NRAv,OK Not To Be OK
197177,4G7uzhxgKFBCJApBpuuBgU,Kabet
197705,3KUGAgxo3b81X5bWoOp0U8,Back Door
197788,0rbKrBvZUYY9GN9l057BuY,"always, i'll care"


In [12]:
len(pd.unique(track_df['track_id'].values)),len(pd.unique(track_df['track_name'].values))

(2292, 1826)

> Q: Why is it that we have fewer unique track names than unique track ids? Is this expected or does it indicate a data processing error?

In [19]:
track_list = track_df['track_id'].values
df_list=[]

for i,track_id in enumerate(track_list):
    print('[%d/%d] Fetching track data for %s... ' % 
          (i+1,len(track_list),track_df[track_df['track_id']==track_id]['track_name'].values[0]), end = " ") 
    track_data = get_track_data(track_id) 
    df_list.append(track_data)
    print('done!')
    
    #sleep for 100 secs per 100 requests to avoid being blocked
    if (i % 100 == 0)&(i > 0):
        time.sleep(100)    

[1/2292] Fetching track data for Havana...  done!
[2/2292] Fetching track data for Perfect...  done!
[3/2292] Fetching track data for What Lovers Do (feat. SZA)...  done!
[4/2292] Fetching track data for Too Good At Goodbyes...  done!
[5/2292] Fetching track data for New Rules...  done!
[6/2292] Fetching track data for Young Dumb & Broke...  done!
[7/2292] Fetching track data for Titibo-Tibo...  done!
[8/2292] Fetching track data for Wolves...  done!
[9/2292] Fetching track data for Super Far...  done!
[10/2292] Fetching track data for Malaya (Camp Sawi Original Motion Picture Soundtrack)...  done!
[11/2292] Fetching track data for Let Me Go (with Alesso, Florida Georgia Line & watt)...  done!
[12/2292] Fetching track data for I Like Me Better...  done!
[13/2292] Fetching track data for Meant to Be (feat. Florida Georgia Line)...  done!
[14/2292] Fetching track data for Hayaan Mo Sila (Inspired by I'm the One)...  done!
[15/2292] Fetching track data for Torete - From "Love You to the S

[123/2292] Fetching track data for Bartier Cardi (feat. 21 Savage)...  done!
[124/2292] Fetching track data for XO Tour Llif3...  done!
[125/2292] Fetching track data for Dying Inside To Hold You - From " All Of You" Official Soundtrack...  done!
[126/2292] Fetching track data for All Night...  done!
[127/2292] Fetching track data for Need You...  done!
[128/2292] Fetching track data for 24K Magic...  done!
[129/2292] Fetching track data for Maybe This Time - From "Maybe This Time"...  done!
[130/2292] Fetching track data for Tell Me Where It Hurts...  done!
[131/2292] Fetching track data for Let Me Be the One...  done!
[132/2292] Fetching track data for Your Song...  done!
[133/2292] Fetching track data for More Than You Know...  done!
[134/2292] Fetching track data for So Sick...  done!
[135/2292] Fetching track data for DNA...  done!
[136/2292] Fetching track data for By Your Side...  done!
[137/2292] Fetching track data for Gorgeous...  done!
[138/2292] Fetching track data for Pari

[250/2292] Fetching track data for Come with Me (feat. Bosx1ne, Flow-G, King Badger & Jroa)...  done!
[251/2292] Fetching track data for Havana (feat. Young Thug)...  done!
[252/2292] Fetching track data for Never Be the Same...  done!
[253/2292] Fetching track data for Real Friends...  done!
[254/2292] Fetching track data for She Loves Control...  done!
[255/2292] Fetching track data for Consequences...  done!
[256/2292] Fetching track data for All These Years...  done!
[257/2292] Fetching track data for Inside Out...  done!
[258/2292] Fetching track data for Something's Gotta Give...  done!
[259/2292] Fetching track data for In the Dark...  done!
[260/2292] Fetching track data for Tell Me...  done!
[261/2292] Fetching track data for Into It...  done!
[262/2292] Fetching track data for Rewrite The Stars...  done!
[263/2292] Fetching track data for Pahinga...  done!
[264/2292] Fetching track data for Cebuana...  done!
[265/2292] Fetching track data for All The Stars (with SZA)...  done

[381/2292] Fetching track data for Next To Me...  done!
[382/2292] Fetching track data for Wait (feat. A Boogie Wit da Hoodie)...  done!
[383/2292] Fetching track data for Like I Do...  done!
[384/2292] Fetching track data for Oks Lang...  done!
[385/2292] Fetching track data for Take Her to the Moon...  done!
[386/2292] Fetching track data for Psycho (feat. Ty Dolla $ign)...  done!
[387/2292] Fetching track data for The Middle...  done!
[388/2292] Fetching track data for Beautiful...  done!
[389/2292] Fetching track data for Thunder...  done!
[390/2292] Fetching track data for Make Me Feel...  done!
[391/2292] Fetching track data for Cebuana...  done!
[392/2292] Fetching track data for Ignition - Remix...  done!
[393/2292] Fetching track data for My Boo...  done!
[394/2292] Fetching track data for Daydream (백일몽)...  done!
[395/2292] Fetching track data for Hope World...  done!
[396/2292] Fetching track data for 항상 (HANGSANG)...  done!
[397/2292] Fetching track data for P.O.P (Piece Of

[518/2292] Fetching track data for rockstar (feat. 21 Savage)...  done!
[519/2292] Fetching track data for Spoil My Night (feat. Swae Lee)...  done!
[520/2292] Fetching track data for Candy Paint...  done!
[521/2292] Fetching track data for Rich & Sad...  done!
[522/2292] Fetching track data for Stay...  done!
[523/2292] Fetching track data for Zack And Codeine...  done!
[524/2292] Fetching track data for Little Do You Know...  done!
[525/2292] Fetching track data for Nadarang - Campfire Mix...  done!
[526/2292] Fetching track data for Have It All...  done!
[527/2292] Fetching track data for There for You...  done!
[528/2292] Fetching track data for Let Me Love You...  done!
[529/2292] Fetching track data for Hide Away...  done!
[530/2292] Fetching track data for Don't Let Me Down (feat. Daya)...  done!
[531/2292] Fetching track data for Work from Home (feat. Ty Dolla $ign)...  done!
[532/2292] Fetching track data for CAN'T STOP THE FEELING! (Original Song from DreamWorks Animation's "

...

[1654/2292] Fetching track data for Call You Mine (feat. Bebe Rexha)...  done!
[1655/2292] Fetching track data for Lover (Remix) [feat. Shawn Mendes]...  done!
[1656/2292] Fetching track data for Yellow Hearts...  done!
[1657/2292] Fetching track data for If the World Was Ending - feat. Julia Michaels...  done!
[1658/2292] Fetching track data for Mean It...  done!
[1659/2292] Fetching track data for Feelings...  done!
[1660/2292] Fetching track data for fuck, i'm lonely (with Anne-Marie) - from “13 Reasons Why: Season 3”...  done!
[1661/2292] Fetching track data for everything i wanted...  done!
[1662/2292] Fetching track data for i'm so tired......  done!
[1663/2292] Fetching track data for Sims...  done!
[1664/2292] Fetching track data for Sad Forever...  done!
[1665/2292] Fetching track data for ROXANNE...  done!
[1666/2292] Fetching track data for Living Proof...  done!
[1667/2292] Fetching track data for Yellow Hearts...  done!
[1668/2292] Fetching track data for Up All Night...  

[1780/2292] Fetching track data for Me & You Together Song...  done!
[1781/2292] Fetching track data for AWIT...  done!
[1782/2292] Fetching track data for Ivana...  done!
[1783/2292] Fetching track data for Godzilla (feat. Juice WRLD)...  done!
[1784/2292] Fetching track data for Lowkey...  done!
[1785/2292] Fetching track data for Any song...  done!
[1786/2292] Fetching track data for Mean It...  done!
[1787/2292] Fetching track data for Feelings...  done!
[1788/2292] Fetching track data for Changes...  done!
[1789/2292] Fetching track data for fuck, i'm lonely - from “13 Reasons Why: Season 3”...  done!
[1790/2292] Fetching track data for i'm so tired......  done!
[1791/2292] Fetching track data for Simmer...  done!
[1792/2292] Fetching track data for Repeat...  done!
[1793/2292] Fetching track data for What If I Told You That I Love You...  done!
[1794/2292] Fetching track data for me & ur ghost...  done!
[1795/2292] Fetching track data for See You Again (feat. Charlie Puth)...  do

[1913/2292] Fetching track data for Here I Am Again...  done!
[1914/2292] Fetching track data for Bounty...  done!
[1915/2292] Fetching track data for Lose Control...  done!
[1916/2292] Fetching track data for Malayo Ka Man...  done!
[1917/2292] Fetching track data for Something New (feat. Ty Dolla $ign)...  done!
[1918/2292] Fetching track data for Landas...  done!
[1919/2292] Fetching track data for Falling...  done!
[1920/2292] Fetching track data for Break My Heart...  done!
[1921/2292] Fetching track data for Teka Lang...  done!
[1922/2292] Fetching track data for Savage...  done!
[1923/2292] Fetching track data for Don't Start Now...  done!
[1924/2292] Fetching track data for Break My Heart...  done!
[1925/2292] Fetching track data for Physical...  done!
[1926/2292] Fetching track data for Paalam...  done!
[1927/2292] Fetching track data for Patawad, Paalam...  done!
[1928/2292] Fetching track data for Future Nostalgia...  done!
[1929/2292] Fetching track data for Levitating...  

[2046/2292] Fetching track data for Ghost...  done!
[2047/2292] Fetching track data for Sour Candy (with BLACKPINK)...  done!
[2048/2292] Fetching track data for 911...  done!
[2049/2292] Fetching track data for Sour Candy (with BLACKPINK)...  done!
[2050/2292] Fetching track data for Rain On Me (with Ariana Grande)...  done!
[2051/2292] Fetching track data for Stupid Love...  done!
[2052/2292] Fetching track data for Free Woman...  done!
[2053/2292] Fetching track data for Alice...  done!
[2054/2292] Fetching track data for Fun Tonight...  done!
[2055/2292] Fetching track data for Lose Somebody...  done!
[2056/2292] Fetching track data for Chromatica II...  done!
[2057/2292] Fetching track data for Enigma...  done!
[2058/2292] Fetching track data for Plastic Doll...  done!
[2059/2292] Fetching track data for Chromatica I...  done!
[2060/2292] Fetching track data for Sine From Above (with Elton John)...  done!
[2061/2292] Fetching track data for Replay...  done!
[2062/2292] Fetching tr

[2177/2292] Fetching track data for Best Song Ever...  done!
[2178/2292] Fetching track data for You & I...  done!
[2179/2292] Fetching track data for One Thing...  done!
[2180/2292] Fetching track data for More Than This...  done!
[2181/2292] Fetching track data for Gotta Be You...  done!
[2182/2292] Fetching track data for Midnight Memories...  done!
[2183/2292] Fetching track data for Last First Kiss...  done!
[2184/2292] Fetching track data for the 1...  done!
[2185/2292] Fetching track data for cardigan...  done!
[2186/2292] Fetching track data for exile (feat. Bon Iver)...  done!
[2187/2292] Fetching track data for my tears ricochet...  done!
[2188/2292] Fetching track data for the last great american dynasty...  done!
[2189/2292] Fetching track data for august...  done!
[2190/2292] Fetching track data for mirrorball...  done!
[2191/2292] Fetching track data for this is me trying...  done!
[2192/2292] Fetching track data for seven...  done!
[2193/2292] Fetching track data for ill

In [20]:
tracks_data_df = pd.concat(df_list)
tracks_data_df.head()

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0ofbQMrRDsUaVKq2mGLEAb,Havana,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,5chBPOVY2I0bG5V3igb5QL,216896,2017-08-03,5,0.768,0.517,7,-4.323,0,0.0312,0.18600,0.000038,0.1040,0.418,104.992
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,3T4tUhGYeRNVUGevb0wThu,263400,2017-03-03,86,0.599,0.448,8,-6.312,1,0.0232,0.16300,0.000000,0.1060,0.168,95.050
0,3hBBKuWJfxlIlnd9QFoC8k,What Lovers Do (feat. SZA),04gDigrS5kc9YWfZHwBETP,Maroon 5,1Jmq5HEJeA9kNi2SgQul4U,199849,2017-11-03,5,0.795,0.615,5,-5.211,0,0.0671,0.07860,0.000003,0.0855,0.393,110.009
0,1mXVgsBdtIVeCLJnSnmtdV,Too Good At Goodbyes,2wY79sveU1sp5g7SokKOiI,Sam Smith,3TJz2UBNYJtlEly0sPeNrQ,201000,2017-11-03,81,0.681,0.372,5,-8.237,1,0.0432,0.64000,0.000000,0.1690,0.476,91.873
0,2ekn2ttSfGqwhhate0LSR0,New Rules,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa,01sfgrNbnnPUEyz6GZYlt9,209320,2017-06-02,81,0.762,0.700,9,-6.021,0,0.0694,0.00261,0.000016,0.1530,0.608,116.073


In [21]:
tracks_data_df.to_csv('data/spotify_daily_charts_tracks.csv', index=False, encoding='utf-8')

In [22]:
tracks_data_df.describe()

,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000,2292.000000
mean,215783.227312,53.542321,0.635396,0.591876,5.123037,-6.876896,0.688045,0.083546,0.286942,0.009645,0.168288,0.455961,118.496587
std,46745.918466,28.083859,0.143064,0.190003,3.593686,2.860197,0.463393,0.083280,0.277537,0.069810,0.126592,0.213066,28.125389
min,37640.000000,0.000000,0.184000,0.054100,0.000000,-24.250000,0.000000,0.023200,0.000002,0.000000,0.021500,0.039800,52.572000
25%,189132.750000,44.000000,0.547000,0.461000,2.000000,-8.291000,0.000000,0.035800,0.053150,0.000000,0.092900,0.288750,96.977750
50%,209273.000000,63.000000,0.653000,0.602000,5.000000,-6.428500,1.000000,0.050950,0.186000,0.000000,0.118000,0.446500,116.004000
75%,238995.500000,74.000000,0.736000,0.738000,8.000000,-4.876250,1.000000,0.090550,0.468000,0.000017,0.200250,0.608000,136.056500
max,536217.000000,100.000000,0.953000,0.969000,11.000000,0.175000,1.000000,0.733000,0.979000,0.908000,0.955000,0.973000,207.476000


## Get data of unique artists in charts 

In [23]:
#Get unique artists id
artist_df = tracks_data_df[['artist_id','artist_name']].drop_duplicates()
artist_df

,artist_id,artist_name
0,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello
0,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran
0,04gDigrS5kc9YWfZHwBETP,Maroon 5
0,2wY79sveU1sp5g7SokKOiI,Sam Smith
0,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa
...,...,...
0,3QJUFtGBGL05vo0kCJZsmT,salem ilese
0,3wVXTWabe3viT0jF7DfjOL,Vedo
0,3hB5x7E5sQzBHuG3xiqmUC,Logan Harris
0,5R4HkjoZdxW3ZoVGC0e0qD,Julian Sean


In [25]:
len(artist_df)

606

> Q: What does the ratio of unique artists to unique tracks tell you about the nature of the Spotify top-streamed market?

In [26]:
def get_artist_data(a_id):
       
    artist_data = sp.artist(a_id)

    ad_list = [a_id,\
               artist_data['name'],\
               artist_data['followers']['total'],\
               artist_data['genres'],\
               artist_data['popularity']]
    data = pd.DataFrame([ad_list], columns = ['artist_id','artist_name','total_followers','genres','popularity'])

    return data


In [27]:
get_artist_data(artist_id)

,artist_id,artist_name,total_followers,genres,popularity
0,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,26727954,"[dance pop, pop]",89


In [28]:
artist_list = artist_df['artist_id'].values
df_list=[]

for i,artist_id in enumerate(artist_list):
    print('[%d/%d] Fetching artist data for %s... ' % 
          (i+1,len(artist_list),artist_df[artist_df['artist_id']==artist_id]['artist_name'].values[0]), end = " ") 
    artist_data = get_artist_data(artist_id) 
    df_list.append(artist_data)
    print('done!')
    
    #sleep for 100 secs per 100 requests to avoid being blocked
    if (i % 100 == 0)& (i > 0):
        time.sleep(100)   

[1/606] Fetching artist data for Camila Cabello...  done!
[2/606] Fetching artist data for Ed Sheeran...  done!
[3/606] Fetching artist data for Maroon 5...  done!
[4/606] Fetching artist data for Sam Smith...  done!
[5/606] Fetching artist data for Dua Lipa...  done!
[6/606] Fetching artist data for Khalid...  done!
[7/606] Fetching artist data for Moira Dela Torre...  done!
[8/606] Fetching artist data for Selena Gomez...  done!
[9/606] Fetching artist data for LANY...  done!
[10/606] Fetching artist data for Hailee Steinfeld...  done!
[11/606] Fetching artist data for Lauv...  done!
[12/606] Fetching artist data for Bebe Rexha...  done!
[13/606] Fetching artist data for Ex Battalion...  done!
[14/606] Fetching artist data for Charlie Puth...  done!
[15/606] Fetching artist data for Jonas Blue...  done!
[16/606] Fetching artist data for Liam Payne...  done!
[17/606] Fetching artist data for NF...  done!
[18/606] Fetching artist data for Maggie Lindemann...  done!
[19/606] Fetching ar

[148/606] Fetching artist data for Drake...  done!
[149/606] Fetching artist data for BLACKPINK...  done!
[150/606] Fetching artist data for Rudimental...  done!
[151/606] Fetching artist data for Cashmere Cat...  done!
[152/606] Fetching artist data for Mayonnaise...  done!
[153/606] Fetching artist data for Boyz II Men...  done!
[154/606] Fetching artist data for Migos...  done!
[155/606] Fetching artist data for Red Velvet...  done!
[156/606] Fetching artist data for Carlie Hanson...  done!
[157/606] Fetching artist data for Bazzi...  done!
[158/606] Fetching artist data for The Weeknd...  done!
[159/606] Fetching artist data for Loren Allred...  done!
[160/606] Fetching artist data for Ziv Zaifman...  done!
[161/606] Fetching artist data for Hugh Jackman...  done!
[162/606] Fetching artist data for IV Of Spades...  done!
[163/606] Fetching artist data for Michelle Williams...  done!
[164/606] Fetching artist data for Austyn Johnson...  done!
[165/606] Fetching artist data for Julia

[294/606] Fetching artist data for Nick Jonas...  done!
[295/606] Fetching artist data for No Rome...  done!
[296/606] Fetching artist data for the bird and the bee...  done!
[297/606] Fetching artist data for Girls' Generation-Oh!GG...  done!
[298/606] Fetching artist data for Silk City...  done!
[299/606] Fetching artist data for Alesso...  done!
[300/606] Fetching artist data for Ciara...  done!
[301/606] Fetching artist data for Khel Pangilinan...  done!
[302/606] Fetching artist data for Lukas Graham...  done!
[303/606] Fetching artist data for Joji...  done!
[304/606] Fetching artist data for Gucci Mane...  done!
[305/606] Fetching artist data for Because...  done!
[306/606] Fetching artist data for Avril Lavigne...  done!
[307/606] Fetching artist data for Lil Peep...  done!
[308/606] Fetching artist data for Leanne & Naara...  done!
[309/606] Fetching artist data for juan karlos...  done!
[310/606] Fetching artist data for Lil Wayne...  done!
[311/606] Fetching artist data for 

[441/606] Fetching artist data for Mena Massoud...  done!
[442/606] Fetching artist data for Naomi Scott...  done!
[443/606] Fetching artist data for Alan Menken...  done!
[444/606] Fetching artist data for Young Thug...  done!
[445/606] Fetching artist data for LEE HI...  done!
[446/606] Fetching artist data for Will Smith...  done!
[447/606] Fetching artist data for SOMI...  done!
[448/606] Fetching artist data for Ali Gatie...  done!
[449/606] Fetching artist data for Sam Feldt...  done!
[450/606] Fetching artist data for Reneé Dominique...  done!
[451/606] Fetching artist data for Donnalyn Bartolome...  done!
[452/606] Fetching artist data for Ashley O...  done!
[453/606] Fetching artist data for J Boog...  done!
[454/606] Fetching artist data for Blanco Brown...  done!
[455/606] Fetching artist data for VVS Collective...  done!
[456/606] Fetching artist data for Arvey...  done!
[457/606] Fetching artist data for King Badger...  done!
[458/606] Fetching artist data for Stephanie Po

[588/606] Fetching artist data for WALK THE MOON...  done!
[589/606] Fetching artist data for TREASURE...  done!
[590/606] Fetching artist data for Sam Kim...  done!
[591/606] Fetching artist data for BØRNS...  done!
[592/606] Fetching artist data for Jem Macatuno...  done!
[593/606] Fetching artist data for Bandang Lapis...  done!
[594/606] Fetching artist data for Tate McRae...  done!
[595/606] Fetching artist data for 24kGoldn...  done!
[596/606] Fetching artist data for Pop Smoke...  done!
[597/606] Fetching artist data for DJ Noiz...  done!
[598/606] Fetching artist data for Shane Filan...  done!
[599/606] Fetching artist data for Jimmy Bondoc...  done!
[600/606] Fetching artist data for Joel Corry...  done!
[601/606] Fetching artist data for Pink Sweat$...  done!
[602/606] Fetching artist data for salem ilese...  done!
[603/606] Fetching artist data for Vedo...  done!
[604/606] Fetching artist data for Logan Harris...  done!
[605/606] Fetching artist data for Julian Sean...  done

In [31]:
artist_data_df = pd.concat(df_list)
artist_data_df 

,artist_id,artist_name,total_followers,genres,popularity
0,4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,18078605,"[dance pop, pop, post-teen pop]",87
0,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,69753264,"[pop, uk pop]",93
0,04gDigrS5kc9YWfZHwBETP,Maroon 5,26892218,"[pop, pop rock]",90
0,2wY79sveU1sp5g7SokKOiI,Sam Smith,13815441,"[pop, post-teen pop, uk pop]",89
0,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa,19536508,"[dance pop, pop, uk pop]",93
...,...,...,...,...,...
0,3QJUFtGBGL05vo0kCJZsmT,salem ilese,36095,[modern indie pop],74
0,3wVXTWabe3viT0jF7DfjOL,Vedo,149172,"[pop r&b, urban contemporary]",73
0,3hB5x7E5sQzBHuG3xiqmUC,Logan Harris,35016,[],63
0,5R4HkjoZdxW3ZoVGC0e0qD,Julian Sean,5541,[opm],50


In [30]:
artist_data_df.to_csv('data/spotify_daily_charts_artists.csv', index=False, encoding='utf-8')

NameError: name 'artist_data_df' is not defined

## Resources
- Spotify API reference manual https://developer.spotify.com/documentation/web-api/reference/search/search/